In [1]:

import numpy as np

from cogent3.maths.matrix_exponential_integration import expected_number_subs
import numpy as np
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [2]:
Q_matrix = np.array([[-1.75025094,  0.94143256,  0.45306226,  0.35575611],
       [ 0.49505807, -2.27893035,  0.88176788,  0.90210439],
       [ 0.36411798,  0.18070926, -0.91594839,  0.37112115],
       [ 0.77758124,  0.75396143,  0.52505328, -2.05659595]])
n = 1000
pi = [0.35, 0.15, 0.05, 0.45]

In [3]:
theoretical_ns_dict = expected_number_subs(pi, Q_matrix, 3)
theoretical_ns_dict

4.974190140095442

In [4]:

repeats = [10, 50, 100, 200, 300, 500]
time_range = [0.5, 1, 2, 3, 4]
repeat = 200

In [5]:
# seed_seq = range(10)
# av_ns = []
# for seed in seed_seq:
#     simulator = SeqSimulate(Q_matrix, n, 100, seed, pi)
#     result = simulator.average_substitution(max_time = 0.3)
#     av_ns.append(result[1])
# av_ns

In [6]:
# simulator = SeqSimulate(Q_matrix, n, 100, seed, pi)
# result = simulator.average_substitution(max_time = 3)
# result[1]

In [7]:

# p_value_repeats = []
# p_value_times = []
# avg_ns_per_site_list = []
# ns_persite_list = []
# for time in time_range:
#     simulator = SeqSimulate(Q_matrix, 1000, repeat, 123, pi)
#     result = simulator.average_substitution(time)
#     ns_per_site1 = result[0]
#     avg_ns_per_site = result[1]
#     avg_ns_per_site_list.append(avg_ns_per_site)
#     ns_persite_list.append(ns_per_site1)
#     theoretical = expected_number_subs(pi, Q_matrix, time)
#     t_stat, p_value = stats.ttest_1samp(ns_per_site1, theoretical)
#     p_value_times.append(p_value)

In [8]:
theoretical_dict = {}
# ns_per_site_dict = {'1000': {}}

for i in range(len(time_range)):
    theoretical_dict[time_range[i]] = expected_number_subs(pi, Q_matrix, time_range[i])
    # ns_per_site_dict['1000'][time_range[i]] = {'ns_per_site_list': ns_persite_list[i], 'avg_ns_per_site': avg_ns_per_site_list[i], 'p_value': p_value_times[i]}


In [14]:
with open('../../../results/WTS_1000.json', 'r') as file:
    data = json.load(file)

In [25]:
def get_histograms2(ns_dict, theoretical_ns_list):
    lengths = list(ns_dict.keys())
    all_times = {time for length in ns_dict for time in ns_dict[length].keys()}
    times = sorted(all_times)  # Sorting to maintain a consistent order

    rows = len(lengths)
    cols = len(times)
    
    # Determine global minimum and maximum x values for axis range consistency
    x_min = min(min(ns_dict[length][time]['ns_per_site_list']) for length in ns_dict for time in ns_dict[length] if time in ns_dict[length])
    x_max = max(max(ns_dict[length][time]['ns_per_site_list']) for length in ns_dict for time in ns_dict[length] if time in ns_dict[length])
    
    # Create subplots
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'Time = {t}, Length = {l}' for l in lengths for t in times])

    # Populate subplots
    for row, length in enumerate(lengths, start=1):
        for col, time in enumerate(times, start=1):
            data = ns_dict[length][time]['ns_per_site_list'] if time in ns_dict[length] else []
            theoretical_ns = theoretical_ns_list[time] if time in theoretical_ns_list else None
            average_ns = ns_dict[length][time]['avg_ns_per_site'] if time in ns_dict[length] else None

            # Add histogram to subplot
            fig.add_trace(
                go.Histogram(
                    x=data,
                    xbins=dict(  # Control the bar widths here
                        start=x_min,
                        end=x_max,
                        size=(x_max - x_min) / 20  # Adjust size for consistent bar width
                    ),
                    marker=dict(line=dict(width=1)),
                    name=f'Length {length}, Time {time}'
                ),
                row=row,
                col=col
            )
            
            # Add vertical lines for average and theoretical values
            fig.add_vline(x=average_ns, line_width=2, line_dash="dash", line_color="red", row=row, col=col)
            fig.add_vline(x=theoretical_ns, line_width=2, line_dash="dash", line_color="green", row=row, col=col)

    # Update layout for all subplots
    fig.update_layout(
        height=300 * rows,
        width=300 * cols,
        showlegend=False,
        bargap=0.05  # Adjust space between bars
    )
    
    # Set consistent x-axis range across all subplots
    for r in range(1, rows+1):
        for c in range(1, cols+1):
            fig.update_xaxes(range=[x_min, x_max], row=r, col=c)
            fig.update_yaxes(range=[0, 60], row=r, col=c)

    return fig


In [15]:
ns_result_dict_1000 = {}
for length, times in data.items():
    ns_result_dict_1000[length] = {float(key): value for key, value in times.items()}

In [22]:
error_ns_dict_all = {}
for key, times in ns_result_dict_1000.items():
    error_ns_dict_all[key] = {}  # Ensures key is initialized
    for time, ns_value in times.items():
        ns_error_list = [a - theoretical_dict[time] for a in ns_value['ns_per_site_list']]
        error_ns_dict_all[key][time] = {
            'ns_per_site_list': ns_error_list,
            'avg_ns_per_site': ns_value['avg_ns_per_site'] - theoretical_dict[time]   
        }

In [23]:
theoretical_ns_error_dict = {time: 0 for time in time_range}

In [26]:
get_histograms2(error_ns_dict_all, theoretical_ns_error_dict)